<a href="https://colab.research.google.com/github/UtkarshAIITB/Summer_Telfer/blob/main/Notebooks/calculation_tone_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
!pip3 install pymupdf
!pip3 install textatistic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [134]:
!apt-get install poppler-utils
!pip install pdf2image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [136]:
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [137]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import fitz
from textatistic import Textatistic
from pdf2image import convert_from_path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
file = open('/content/drive/MyDrive/Telfer/positive-words.txt', 'r')
pos_words = file.read().split()

In [140]:
file = open('/content/drive/MyDrive/Telfer/negative-words.txt', 'r')
neg_words = file.read().split()

In [141]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1KeJOce_VmmpBKxrNzmnhSGvhJYj84iShQFd5v8QV_BU/edit#gid=0'
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
# print(url_1)
shifters = pd.read_csv(url_1)
shifters.head()

,Word,Classification,Weight
0,absolutely,amplifier,0.8
1,acute,amplifier,0.8
2,acutely,amplifier,0.8
3,almost,de-amplifier,0.8
4,although,adversative-conjuction,0.8


In [142]:
shifters.loc[71, 'Word'] = 'true'

In [143]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
shifters['class_encode'] = label_encoder.fit_transform(shifters['Classification'])
shifters['class_encode'].unique()

array([1, 2, 0, 3])

In [144]:
# amplifier = 1
# de-amplifier = 2
# adversative-conjuction = 0
# negator = 3

In [145]:
def data(path):
  pages = convert_from_path(path, 350)
  text = ''
  i = 1
  for page in pages:
    extractedInformation = pytesseract.image_to_string(page)
      # print(extractedInformation)
    text = text + extractedInformation
  text = text.replace('\n', ' ')
  lines = text.split('.')
  all_reviews = list()
  t_f = ''

  for text in lines:
    stop_words = stopwords.words('english')
    corp = text.lower()
    corp = corp.replace("won't", "wont").replace("weren't", "werent").replace("don't", "dont").replace("doesn't", "doesnt").replace("can't", "cant")
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip()
    tokens = word_tokenize(corp)
    # words = [t for t in tokens if t not in stop_words]
    words = ' '.join(tokens)
    t_f += words
    t_f += '. '
    all_reviews.append(words)
  return all_reviews

In [146]:
# stop_words = stopwords.words('english')
# stop_words

In [147]:
# fin = data('/content/B S Refrigerators Ltd..pdf')

In [148]:
# fin

In [149]:
def paramters(sentences):
  rows = []
  for pp in sentences:
    stop_words = stopwords.words('english')
    corp = pp.lower()
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip()
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in stop_words]
    # lemmatize = [lemma.lemmatize(w) for w in words]
    ind = index_of_valence_shifters(tokens)
    tone = tone_calculator(ind, tokens)
    rows.append([pp, tone])
  
  df = pd.DataFrame(rows, columns=['Sentence', 'Tone'])
  return df

In [150]:
def index_of_valence_shifters(tokens):
  count = 0
  ind = []
  for wor in tokens:
    for valence in shifters['Word']:
      if wor == valence:
        ind.append(tokens.index(valence))
        count +=1
  ind.append(len(tokens))
  return ind

In [151]:
def tone_calculator(ind, tokens):
  tone = 0
  sum = len(ind)-1
  for i in range(0, len(ind)-1):
    pos_count = 0
    neg_count = 0
    sub = tokens[ind[i]:ind[i+1]]
    # print(sub)
    for wor in sub:
      for plus in pos_words:
        if(wor == plus):
          pos_count +=1
          r1 = shifters.loc[shifters['Word'] == sub[0]]
          if(r1.iloc[0]['class_encode'] == 1):
            pos_count = pos_count + r1.iloc[0]['Weight']
          if (r1.iloc[0]['class_encode'] == 2):
            pos_count = pos_count - r1.iloc[0]['Weight']
          if (r1.iloc[0]['class_encode'] == 0):
            pos_count = pos_count + r1.iloc[0]['Weight']
          if (r1.iloc[0]['class_encode'] == 3):
            pos_count = pos_count - r1.iloc[0]['Weight']

    for wor in sub:
      for neg in neg_words:
        if(wor == neg):
          neg_count-=1
          r1 = shifters.loc[shifters['Word'] == sub[0]]
          if(r1.iloc[0]['class_encode'] == 1):
            neg_count = neg_count - r1.iloc[0]['Weight']
          if(r1.iloc[0]['class_encode'] == 2):
            neg_count = neg_count + r1.iloc[0]['Weight']
          if(r1.iloc[0]['class_encode'] == 0):
            neg_count = neg_count - r1.iloc[0]['Weight']
          if (r1.iloc[0]['class_encode'] == 3):
            pos_count = pos_count - r1.iloc[0]['Weight']
      # print('The negative count is: '  + str(neg_count))

      
    stop_words = stopwords.words('english')
    words = [t for t in sub if t not in stop_words]
    sum += len(words)
    # print(sum)
    tone += (pos_count + neg_count)
    # print(tone)
  if (sum!=0):
    tone = tone/sum
  return tone

In [152]:
# fin = data('/content/Anil Ltd..pdf')
# out = paramters(fin)
# use = out[out['Tone']!=0]
# use = use.reset_index(drop = True)

In [153]:
# tone_df = pd.DataFrame(columns = ['Company', 'Tone_sum', 'Tone_avg'])

In [154]:
# use

In [155]:
# use.shape

In [156]:
# tone_sum = use['Tone'].sum()

In [157]:
# tone_sum

In [158]:
# tone_avg = tone_sum/use.shape[0]

In [159]:
# tone_avg

In [160]:
# tone_df.append(['Anil Ltd..pdf', tone_sum, tone_avg])
# new_row = {'name':'Geo', 'physics':87, 'chemistry':92, 'algebra':97}
# new_row = {'Company': 'Anil Ltd..pdf', 'Tone_avg':tone_avg, 'Tone_sum': tone_sum}
# tone_df = tone_df.append(new_row, ignore_index=True)
# tone_df

In [161]:
tone_df = pd.DataFrame(columns = ['Company', 'Tone_sum', 'Tone_avg'])

In [162]:
import os
basepath = '/content/drive/MyDrive/Telfer/report6/'
for filename in os.listdir(basepath):
  if filename.endswith('.pdf'):
    # print(os.path.join("/content/drive/MyDrive/prowess/", filename))
    fin = data(os.path.join("/content/drive/MyDrive/Telfer/report6/", filename))
    out = paramters(fin)
    use = out[out['Tone']!=0]
    use = use.reset_index(drop = True)
    tone_sum = use['Tone'].sum()
    tone_avg = tone_sum/use.shape[0]
    new_row = {'Company': filename, 'Tone_sum': tone_sum , 'Tone_avg':tone_avg}
    tone_df = tone_df.append(new_row, ignore_index=True)

In [163]:
# out
tone_df

,Company,Tone_sum,Tone_avg
0,Blue Star Ltd..pdf,3.834853,0.191743
1,Biocon Ltd..pdf,8.640085,0.172802
2,Bata India Ltd..pdf,2.981020,0.198735
3,Birla Corporation Ltd..pdf,1.545405,0.039626
4,Blue Dart Express Ltd..pdf,5.503603,0.127991
5,Bharti Airtel Ltd..pdf,6.791897,0.081830
6,Bharat Dynamics Ltd..pdf,1.573373,0.104892
7,Berger Paints India Ltd..pdf,4.083844,0.151253
8,Bayer Cropscience Ltd..pdf,3.629285,0.172823
9,Bharat Electronics Ltd..pdf,1.763622,0.110226


In [164]:
tone_df.to_csv('/content/drive/MyDrive/Telfer/Final_datasets/report6.csv')

In [165]:
# out[out['Tone']!=0]

In [166]:
# fin

In [167]:
# tt = 'all consumption indicators except essentials were severely impacted as economic and medical uncertainty will keep consumption weak for some quarters in fy'
# corp = tt.lower()
# corp

In [168]:
# corp = re.sub('[^a-zA-Z]+',' ', corp).strip()
# corp

In [169]:
# tokens = word_tokenize(corp)

In [170]:
# tokens

In [171]:
# ind = []
# for wor in tokens:
#   for valence in shifters['Word']:
#     if wor == valence:
#       ind.append(tokens.index(valence))
#       # count +=1
#   # return ind
# print(ind)

In [172]:
# tone = 0
# sum = len(ind)-1
# for i in range(0, len(ind)-1):
#   pos_count = 0
#   neg_count = 0
#   sub = tokens[ind[i]:ind[i+1]]
#   # print(sub)
#   for wor in sub:
#     for plus in pos_words:
#       if(wor == plus):
#         pos_count +=1
#         r1 = shifters.loc[shifters['Word'] == sub[0]]
#         if(r1.iloc[0]['class_encode'] == 1):
#           pos_count = pos_count + r1.iloc[0]['Weight']
#         if (r1.iloc[0]['class_encode'] == 2):
#           pos_count = pos_count - r1.iloc[0]['Weight']
#         if (r1.iloc[0]['class_encode'] == 0):
#           pos_count = pos_count + r1.iloc[0]['Weight']
#         if (r1.iloc[0]['class_encode'] == 3):
#           pos_count = pos_count - r1.iloc[0]['Weight']

#   for wor in sub:
#     for neg in neg_words:
#       if(wor == neg):
#         neg_count-=1
#         r1 = shifters.loc[shifters['Word'] == sub[0]]
#         if(r1.iloc[0]['class_encode'] == 1):
#           neg_count = neg_count - r1.iloc[0]['Weight']
#         if(r1.iloc[0]['class_encode'] == 2):
#           neg_count = neg_count + r1.iloc[0]['Weight']
#         if(r1.iloc[0]['class_encode'] == 0):
#           neg_count = neg_count - r1.iloc[0]['Weight']
#         if (r1.iloc[0]['class_encode'] == 3):
#           pos_count = pos_count - r1.iloc[0]['Weight']
#     # print('The negative count is: '  + str(neg_count))

    
#   stop_words = stopwords.words('english')
#   words = [t for t in sub if t not in stop_words]
#   sum += len(words)
#   # print(sum)
#   tone += (pos_count + neg_count)
#   # print(tone)
# if (sum!=0):
#   tone = tone/sum
# # return tone
# print(tone)

In [173]:
# for i in range(0, len(ind)):
#   print(i)

In [174]:
# out.to_csv('/content/drive/MyDrive/Telfer/New_csv/y_dis.csv')

In [175]:
# pages = convert_from_path('/content/Advance Powerinfra Tech Ltd..pdf', 350)
# text = ''
# i = 1
# for page in pages:
#   extractedInformation = pytesseract.image_to_string(page)
#     # print(extractedInformation)
#   text = text + extractedInformation
# text = text.replace('\n', ' ')

In [176]:
# from pdf2image import convert_from_path

# pdfs = r"/content/Advance Powerinfra Tech Ltd..pdf"
# pages = convert_from_path(pdfs, 350)
# text = ''
# i = 1
# for page in pages:
#   extractedInformation = pytesseract.image_to_string(page)
#     # print(extractedInformation)
#   text = text + extractedInformation